In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import tkinter as tk
from tkinter import *
import pandas as pd
import datetime
import time
from datetime import datetime, timedelta
from parinya import LINE
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import cv2
import os
line = LINE('iGgobiz5gA1KGNwsJYea0ROgpgFRXPvqRoRuTCdEKCR')

In [2]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/Top/Desktop/python/Chatbot/Bake a Wish bot/Project/bakeawishcredentials.json', scope)
# authorize the clientsheet 
client = gspread.authorize(creds)

In [3]:
sheet = client.open("BaK Stock  Aug-23").worksheet("Stock")
data = sheet.get_values()
df = pd.DataFrame(data)
df.columns = df.loc[1]
df.drop([0,1])

1,group,No,name,thai,c/f,31/12,31/12,31/12,31/12,31/12,...,30/08,30/08,31/08,31/08,01/12,31/08,31/08,31/08,31/08,31/08
2,,,,,,30/06,,,,,...,,,31/08,,,,,,,
3,Group,No,name,name,C/F,รับ,BBF,ขาย,ทิ้ง,ปิดร้าน,...,30-Aug,Loss,รับ,BBF,,ขาย,ทิ้ง,ปิดร้าน,31-Aug,Loss
4,,,Cake Big,,,,,,,,...,,,,,0,,,,,
5,Cake Big,1,Chinese New year Orange,เค้กส้ม,,,06/02,,,-,...,-,-,,10/08,,,,-,-,-
6,Cake Big,2,Red Velvet Cake,เรดเวลใหญ่,,,06/02,,,-,...,-,-,,10/08,,,,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,Macaron,10,Macaron Matcha,,,,06/02,,,,...,-,-,,07/08,,,,-,-,-
177,Macaron,10,mc2,,,,,,,,...,-,-,,,,,,-,-,-
178,Macaron,10,mc3,,,,,,,,...,-,-,,,,,,-,-,-
179,Macaron,10,mc4,,,,,,,,...,-,-,,,,,,-,-,-


In [4]:
#กำหนดตัวแปรวันที่
timestamp = datetime.now()
tomorrow = timestamp + timedelta(1)
yesterday = timestamp - timedelta(1)
next2day = timestamp + timedelta(2)
next3day = timestamp + timedelta(3)
nday = tomorrow.strftime("%d/%m")
n2day = next2day.strftime("%d/%m")
n3day = next3day.strftime("%d/%m")
#print(df)
#date_time = datetime.fromtimestamp(timestamp)
today = timestamp.strftime("%d-%b")
yes = yesterday.strftime("%d-%b")
focusday = timestamp.strftime("%d/%m")
focusday1 = yesterday.strftime("%d/%m")
tomorrow1 = tomorrow.strftime("%d/%m")
#print(focusday1)
#print(focusday)
#print(tomorrow1)
#กำหนดเวลาการใช้งานโปรแกรม 21:00-03:00 นับเป็นวันเก่า
hour = timestamp.strftime("%H")
if float(hour) < 6:
    timestamp = yesterday
    tomorrow = timestamp + timedelta(1)
    yesterday = timestamp - timedelta(1)
    next2day = timestamp + timedelta(2)
    next3day = timestamp + timedelta(3)
    nday = tomorrow.strftime("%d/%m")
    n2day = next2day.strftime("%d/%m")
    n3day = next3day.strftime("%d/%m")
    today = timestamp.strftime("%d-%b")
    yes = yesterday.strftime("%d-%b")
    focusday = timestamp.strftime("%d/%m")
    focusday1 = yesterday.strftime("%d/%m")
    tomorrow1 = tomorrow.strftime("%d/%m")

In [5]:
focus = df[['name',focusday]]
focus.columns = focus.loc[3]
focusm = focus.loc[4:]
focus1 = df[['name',focusday1]]
focus1.columns = focus1.loc[3]
focus1m = focus1.loc[4:]
bbfday = tomorrow1
focusm['รับ'] = focusm['รับ'].replace([''], '0')
#print(focusm)
#print(focus1m)
df1 = focusm[['รับ','name','BBF',today]]
df2 = focus1m[['รับ','name','BBF',yes]]
df2.rename(columns = {'BBF':'BBFO'}, inplace = True)
df1 = pd.concat([df1,df2[yes]], axis=1)
df1 = pd.concat([df1,df2['BBFO']],axis=1)
df1 = df1.replace(['-'], '0')
#print(df1)
#กำหนดค่าในการค้นหา
filt = (df1['BBF'] == bbfday) & (df1['รับ'] != "0") #เช็คว่าของเข้าวันนี้รึเปล่า
filt2 = (df1['BBF'] == bbfday) & (df1[today] != "0") #BBFพรุ่งนี้ 
filt3 = (df1['BBF'] == n2day) & (df1[today] != "0") #BBF 2วัน
filt4 = (df1['BBF'] == n3day) & (df1[today] != "0") #BBF 3วัน
# Source Data
checkdelivery = df1.loc[filt]
exp1 = df1.loc[filt2]
exp2 = df1.loc[filt3]
exp3 = df1.loc[filt4]
checkdelivery.reset_index(drop=True, inplace=True)
exp1.reset_index(drop=True, inplace=True)
exp2.reset_index(drop=True, inplace=True)
exp3.reset_index(drop=True, inplace=True)
#แปลงตารางไว้หาข้อมูล
checkdelivery[yes] = checkdelivery[yes].replace(['-'], '0')
checkdelivery = checkdelivery.copy()
checkdelivery = checkdelivery.astype({'รับ': int,today: int,yes: int})
checkdelivery['test'] = (checkdelivery['รับ'] - checkdelivery[today])

#ฟังก์ชั่นส่งข้อมูลเข้าไลน์ x = ตารางข้อมูล ,  y = วันBBF ที่ต้องการ 
def convertforline(df,x,output_image_path='dummy') :
    df = df.rename(columns={"name":"1",today:"2"})
    df = df[["1","2"]]

    # Plot the DataFrame as a table
    plt.figure(figsize=(8, 3))
    plt.axis('off')
    plt.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='upper center')
    #plt.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')

    # Save the image
    plt.savefig(output_image_path)

    # Close the plot to release resources
    plt.close()
    image1 = cv2.imread(output_image_path+'.png')
    line.sendimage(image1,msg = 'bbfวันที่'+x)
    os.remove(output_image_path+'.png')

def convertforlineSND(df,x,output_image_path='dummy') :
    df = df.rename(columns={"name":"1",test:"2"})
    df = df[["2","1"]]
    plt.figure(figsize=(8, 3))
    plt.axis('off')
    plt.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='upper center')
    #plt.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')

    # Save the image
    plt.savefig(output_image_path)

    # Close the plot to release resources
    plt.close()
    image1 = cv2.imread(output_image_path+'.png')
    line.sendimage(image1,msg = 'bbfวันที่'+x)
    os.remove(output_image_path+'.png')
#CASE 1 ไม่มีของเข้า
if checkdelivery.shape[0] == 0:
    if exp1.shape[0] == 0:
        line.sendtext("BBF>>>> " + str(bbfday) +  ">>>>>ไม่มี  ")
    else:
        convertforline(exp1,bbfday)
    if exp2.shape[0] == 0:
        line.sendtext("BBF>>>> " + str(n2day) +  ">>>>>ไม่มี  ")
    else:
        convertforline(exp2,n2day)

#CASE 2 มีของเข้า
else:
#Step 1 Checkของเก่า BBFO
    case2 = checkdelivery
    filtertest = (case2['test'] < 0) #ขายไม่หมด
    filtertest2 = (case2['BBFO'] == bbfday) & (case2[today] != "0")
    filtertest3 = (case2['BBFO'] == n2day) & (case2[today] != "0") #BBF 2วัน
    sell_not_done = cast2.loc[filtertest]
    sell_not_done['test'] = sell_not_done['test'].abs
    SNDtb = sell_not_done['name','test','BBFO']
    SNDtb2 = sell_not_done[filtertest2]
    SNDtb3 = sell_not_done[filtertest3]
    convertforlineSND(SNDtb2,bbfday)
    convertforlineSND(SNDtb3,n2day)
#Step 2 Checkของใหม่ BBF
    filterbbf = (case2['BBFO'] == bbfday) & (case2[today] != "0")
    filterbbf2 = (case2['BBFO'] == n2day) & (case2[today] != "0")
    Ntb = case2['name',today,'BBF']
    Ntb2 = Ntb[filterbbf]
    Ntb3 = Ntb[filterbbf2]
    convertforline(Ntb2,bbfday)
    
    convertforline(Ntb3,n2day)



KeyError: 'รับ'

In [20]:
df1[df1['BBF']=='04/08']

,รับ,name,BBF,03-Aug,02-Aug,BBFO
57,0,Banoffee Mini,04/08,0,4,04/08
64,0,Joshua,04/08,0,4,04/08
65,0,Panacotta Strawberry ( Mini ),04/08,0,3,04/08
81,0,Tiramisu (Cup),04/08,0,2,04/08
89,0,Custard pudding,04/08,0,5,04/08
163,0,Cream Cone,04/08,0,0,04/08


In [37]:
focus

3,name,รับ,BBF,ขาย,ทิ้ง,ปิดร้าน,03-Aug,Loss
0,,,,,,,,
1,name,03/08,03/08,03/08,03/08,03/08,03/08,03/08
2,,03/08,,,,,,
3,name,รับ,BBF,ขาย,ทิ้ง,ปิดร้าน,03-Aug,Loss
4,Cake Big,,,,,,,
...,...,...,...,...,...,...,...,...
176,Macaron Matcha,,07/08,,,24,-,-24
177,mc2,,,,,-,-,-
178,mc3,,,,,-,-,-
179,mc4,,,,,-,-,-


In [29]:
checkdelivery.shape[0]

0

In [9]:
focusm

3,name,,BBF,ขาย,ทิ้ง,ปิดร้าน,06-Aug,Loss
4,Cake Big,,,1,,,,
5,Chinese New year Orange,,10/08,3,,3,3,-
6,Red Velvet Cake,,10/08,-,,4,4,-
7,Fruit Cake,,,-,,-,-,-
8,Banoffee,,07/08,3,,-3,2,5
...,...,...,...,...,...,...,...,...
176,Macaron Matcha,,07/08,1,,36,34,-2
177,mc2,,,-,,-,-,-
178,mc3,,,-,,-,-,-
179,mc4,,,-,,-,-,-


In [49]:
timestamp = yesterday
tomorrow = timestamp + timedelta(1)
yesterday = timestamp - timedelta(1)
next2day = timestamp + timedelta(2)
next3day = timestamp + timedelta(3)
nday = tomorrow.strftime("%d/%m")
n2day = next2day.strftime("%d/%m")
n3day = next3day.strftime("%d/%m")
today = timestamp.strftime("%d-%b")
yes = yesterday.strftime("%d-%b")
focusday = timestamp.strftime("%d/%m")
focusday1 = yesterday.strftime("%d/%m")
tomorrow1 = tomorrow.strftime("%d/%m")

In [50]:
today

'02-Aug'

In [15]:
focus

3,name,,BBF,ขาย,ทิ้ง,ปิดร้าน,06-Aug,Loss
0,,,,,,,,
1,name,06/08,06/08,06/08,06/08,06/08,06/08,06/08
2,,06/08,,,,,,
3,name,,BBF,ขาย,ทิ้ง,ปิดร้าน,06-Aug,Loss
4,Cake Big,,,1,,,,
...,...,...,...,...,...,...,...,...
176,Macaron Matcha,,07/08,1,,36,34,-2
177,mc2,,,-,,-,-,-
178,mc3,,,-,,-,-,-
179,mc4,,,-,,-,-,-
